# Lab3 - Assignment Sentiment

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook describes the LAB-2 assignment of the Text Mining course. It is about sentiment analysis.

The aims of the assignment are:
* Learn how to run a rule-based sentiment analysis module (VADER)
* Learn how to run a machine learning sentiment analysis module (Scikit-Learn/ Naive Bayes)
* Learn how to run scikit-learn metrics for the quantitative evaluation
* Learn how to perform and interpret a quantitative evaluation of the outcomes of the tools (in terms of Precision, Recall, and F<sub>1</sub>)
* Learn how to evaluate the results qualitatively (by examining the data) 
* Get insight into differences between the two applied methods
* Get insight into the effects of using linguistic preprocessing
* Be able to describe differences between the two methods in terms of their results
* Get insight into issues when applying these methods across different  domains

In this assignment, you are going to create your own gold standard set from 50 tweets. You will the VADER and scikit-learn classifiers to these tweets and evaluate the results by using evaluation metrics and inspecting the data.

We recommend you go through the notebooks in the following order:
* **Read the assignment (see below)**
* **Lab3.2-Sentiment-analysis-with-VADER.ipynb**
* **Lab3.3-Sentiment-analysis.with-scikit-learn.ipynb**
* **Answer the questions of the assignment (see below) using the provided notebooks and submit**

In this assignment you are asked to perform both quantitative evaluations and error analyses:
* a quantitative evaluation concerns the scores (Precision, Recall, and F<sub>1</sub>) provided by scikit's classification_report. It includes the scores per category, as well as micro and macro averages. Discuss whether the scores are balanced or not between the different categories (positive, negative, neutral) and between precision and recall. Discuss the shortcomings (if any) of the classifier based on these scores
* an error analysis regarding the misclassifications of the classifier. It involves going through the texts and trying to understand what has gone wrong. It servers to get insight in what could be done to improve the performance of the classifier. Do you observe patterns in misclassifications?  Discuss why these errors are made and propose ways to solve them.

## Credits
The notebooks in this block have been originally created by [Marten Postma](https://martenpostma.github.io) and [Isa Maks](https://research.vu.nl/en/persons/e-maks). Adaptations were made by [Filip Ilievski](http://ilievski.nl).

## Part I: VADER assignments


### Preparation (nothing to submit):
To be able to answer the VADER questions you need to know how the tool works. 
* Read more about the VADER tool in [this blog](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html).  
* VADER provides 4 scores (positive, negative, neutral, compound). Be sure to understand what they mean and how they are calculated.
* VADER uses rules to handle linguistic phenomena such as negation and intensification. Be sure to understand which rules are used, how they work, and why they are important.
* VADER makes use of a sentiment lexicon. Have a look at the lexicon. Be sure to understand which information can be found there (lemma?, wordform?, part-of-speech?, polarity value?, word meaning?) What do all scores mean? https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt) 


### [3.5 points] Question1:

Regard the following sentences and their output as given by VADER. Regard sentences 1 to 7, and explain the outcome **for each sentence**. Take into account both the rules applied by VADER and the lexicon that is used. You will find that some of the results are reasonable, but others are not. Explain what is going wrong or not when correct and incorrect results are produced. 

```
INPUT SENTENCE 1 I love apples
VADER OUTPUT {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

INPUT SENTENCE 2 I don't love apples
VADER OUTPUT {'neg': 0.627, 'neu': 0.373, 'pos': 0.0, 'compound': -0.5216}

INPUT SENTENCE 3 I love apples :-)
VADER OUTPUT {'neg': 0.0, 'neu': 0.133, 'pos': 0.867, 'compound': 0.7579}

INPUT SENTENCE 4 These houses are ruins
VADER OUTPUT {'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}

INPUT SENTENCE 5 These houses are certainly not considered ruins
VADER OUTPUT {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.5867}

INPUT SENTENCE 6 He lies in the chair in the garden
VADER OUTPUT {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4215}

INPUT SENTENCE 7 This house is like any house
VADER OUTPUT {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.3612}
```

**answers:**
* sentence 1: No negative words, so neg is 0.0. Other words are neutral or positive so results in a postive compound score based on how postive the words are.
* sentence 2: Large portion of the sentence is seen as negative because of 'don't', it is strange that because don't was added now 63% of the sentence is negative. The overall score could have been higher since don't love is not very negative.
* sentence 3: Score is higher than sentence 1, because ':-)' with a lexicon of 1.3 adds a lot of positivity to the sentence.
* sentence 4: 'ruins' doesn't necessarily mean that is is negative. It can be monumental as well. It should be more neutral than negative. Should be based on context and not just the word itself. ('ruins' has a score of -1.9)
* sentence 5: This is a clear example of the negation used by VADER. 'not ruins' is seen as positive.
* sentence 6: 'lies' is seen as negative, probably because people annotaded the word as negative because of the association with someone lying and not someone lying down. Doesn't really take into account what the word means in different contexts.
* sentence 7: Probably more on the positive side because of the word like, but would have been more logical if it was 100% neutral, since it is a statement and doesn't say anything about the house.



### [Points: 2.5] Exercise 2: Collecting 50 tweets for evaluation
Collect 50 tweets. Try to find tweets that are interesting for sentiment analysis, e.g., very positive, neutral, and negative tweets. 

We will store the tweets in the file **my_tweets.json** (use a text editor to edit).
For each tweet, you should insert:
* sentiment analysis label: negative | neutral | positive (this you determine yourself, this is not done by a computer)
* the text of the tweet
* the Tweet-URL

from:
```
    "1": {
        "sentiment_label": "",
        "text_of_tweet": "",
        "tweet_url": "",
```
to:
```
"1": {
        "sentiment_label": "positive",
        "text_of_tweet": "All across America people chose to get involved, get engaged and stand up. Each of us can make a difference, and all of us ought to try. So go keep changing the world in 2018.",
        "tweet_url" : "https://twitter.com/BarackObama/status/946775615893655552",
    },
```

You can load your tweets with human annotation in the following way.

In [10]:
import json

In [11]:
my_tweets = json.load(open('my_tweets.json'))

In [12]:
for id_, tweet_info in my_tweets.items():
    print(id_, tweet_info)
    break

1 {'sentiment_label': 'negative', 'text_of_tweet': 'They spied on my campaign!', 'tweet_url': 'https://twitter.com/realDonaldTrump/status/1232908958421131264'}


### [5 points] Question 3:

Run VADER on your own tweets (see function **run_vader** from notebook **Lab2-Sentiment-analysis-using-VADER.ipynb**). You can use the code snippet below this explanation as a starting point. 
* [2.5 points] a. Perform a quantitative evaluation. Explain the different scores, and explain which scores are most relevant and why.
* [2.5 points] b. Perform an error analysis: select 10 positive, 10 negative and 10 neutral tweets that are not correctly classified and try to understand why. Refer to the VADER-rules and the VADER-lexicon. Of course, if there are less than 10 errors for a category, you only have to check those. For example, if there are only 5 errors for positive tweets, you just describe those.

In [13]:
def vader_output_to_label(vader_output):
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
def vader_output_to_label_num(vader_output):
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 0
    elif compound == 0.0:
        return 1
    elif compound > 0.0:
        return 2
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [14]:
tweets = []
all_vader_output = []
gold = []

# settings (to change for different experiments)
to_lemmatize = True 
pos = set()

for id_, tweet_info in my_tweets.items():
    the_tweet = tweet_info['text_of_tweet']
    vader_output = run_vader(the_tweet)
    vader_label = vader_output_to_label(vader_output)
    
    tweets.append(the_tweet)
    all_vader_output.append(vader_label)
    gold.append(tweet_info['sentiment_label'])
    
# use scikit-learn's classification report
print(classification_report(gold, all_vader_output))

              precision    recall  f1-score   support

    negative       0.94      0.73      0.82        22
     neutral       0.60      0.55      0.57        11
    positive       0.61      0.82      0.70        17

    accuracy                           0.72        50
   macro avg       0.72      0.70      0.70        50
weighted avg       0.75      0.72      0.72        50



**answers:**

### [4 points] Question 4:
Run VADER on the set of airline tweets with the following settings:

* Run VADER (as it is) on the set of airline tweets 
* Run VADER on the set of airline tweets after having lemmatized the text
* Run VADER on the set of airline tweets with only adjectives
* Run VADER on the set of airline tweets with only adjectives and after having lemmatized the text
* Run VADER on the set of airline tweets with only nouns
* Run VADER on the set of airline tweets with only nouns and after having lemmatized the text
* Run VADER on the set of airline tweets with only verbs
* Run VADER on the set of airline tweets with only verbs and after having lemmatized the text

* [1 point] a. Generate for all separate experiments the classification report, i.e., Precision, Recall, and F<sub>1</sub> scores per category as well as micro and macro averages. **Use a different code cell (or multiple code cells) for each experiment.**
* [3 points] b. Compare the scores and explain what they tell you.
* - Does lemmatisation help? Explain why or why not.
* - Are all parts of speech equally important for sentiment analysis? Explain why or why not.

In [6]:
import pathlib
import nltk
import spacy
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.datasets import load_files
from sklearn.metrics import classification_report

vader_model = SentimentIntensityAnalyzer()
nlp = spacy.load('en') # en_core_web_sm

cwd = pathlib.Path.cwd()
airline_tweets_folder = cwd.joinpath('airlinetweets')
airline_tweets_train = load_files(str(airline_tweets_folder))

data = airline_tweets_train.data
# 0:negative, 1:neutral, 2:positive
labels = airline_tweets_train.target

In [7]:
def run_vader(textual_unit, 
              lemmatize=False, 
              parts_of_speech_to_consider=set(),
              verbose=0):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -empty set -> all parts of speech are provided
    -non-empty set: only these parts of speech are considered
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    doc = nlp(textual_unit)
        
    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

In [66]:
# Run VADER (as it is) on the set of airline tweets
eval_v = [vader_output_to_label_num(run_vader(str(item))) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.80      0.51      0.63      1750
           1       0.60      0.51      0.55      1515
           2       0.56      0.88      0.68      1490

    accuracy                           0.63      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.66      0.63      0.62      4755



In [67]:
# Run VADER on the set of airline tweets after having lemmatized the text
eval_v = [vader_output_to_label_num(run_vader(str(item), lemmatize=True)) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.79      0.52      0.63      1750
           1       0.60      0.49      0.54      1515
           2       0.56      0.87      0.68      1490

    accuracy                           0.62      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.65      0.62      0.62      4755



In [68]:
#Run VADER on the set of airline tweets with only adjectives
eval_v = [vader_output_to_label_num(run_vader(str(item), parts_of_speech_to_consider={'ADJ'})) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1750
           1       0.32      1.00      0.48      1515
           2       0.00      0.00      0.00      1490

    accuracy                           0.32      4755
   macro avg       0.11      0.33      0.16      4755
weighted avg       0.10      0.32      0.15      4755



/Users/sebastiaanvergunst/anaconda3/envs/text_mining/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [69]:
#Run VADER on the set of airline tweets with only adjectives and after having lemmatized the text
eval_v = [vader_output_to_label_num(run_vader(str(item), 
                                              parts_of_speech_to_consider={'ADJ'},
                                              lemmatize=True)) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1750
           1       0.32      1.00      0.48      1515
           2       0.00      0.00      0.00      1490

    accuracy                           0.32      4755
   macro avg       0.11      0.33      0.16      4755
weighted avg       0.10      0.32      0.15      4755



In [70]:
#Run VADER on the set of airline tweets with only nouns
eval_v = [vader_output_to_label_num(run_vader(str(item), parts_of_speech_to_consider={'NOUN'})) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1750
           1       0.32      1.00      0.48      1515
           2       0.00      0.00      0.00      1490

    accuracy                           0.32      4755
   macro avg       0.11      0.33      0.16      4755
weighted avg       0.10      0.32      0.15      4755



In [71]:
#Run VADER on the set of airline tweets with only nouns and after having lemmatized the text
eval_v = [vader_output_to_label_num(run_vader(str(item), 
                                              parts_of_speech_to_consider={'NOUN'},
                                              lemmatize=True)) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1750
           1       0.32      1.00      0.48      1515
           2       0.00      0.00      0.00      1490

    accuracy                           0.32      4755
   macro avg       0.11      0.33      0.16      4755
weighted avg       0.10      0.32      0.15      4755



In [72]:
#Run VADER on the set of airline tweets with only verbs
eval_v = [vader_output_to_label_num(run_vader(str(item), parts_of_speech_to_consider={'VERB'})) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1750
           1       0.32      1.00      0.48      1515
           2       0.00      0.00      0.00      1490

    accuracy                           0.32      4755
   macro avg       0.11      0.33      0.16      4755
weighted avg       0.10      0.32      0.15      4755



In [73]:
#Run VADER on the set of airline tweets with only verbs and after having lemmatized the text
eval_v = [vader_output_to_label_num(run_vader(str(item), 
                                              parts_of_speech_to_consider={'VERB'},
                                              lemmatize=True)) for tweet in data]
print(classification_report(labels, eval_v))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1750
           1       0.32      1.00      0.48      1515
           2       0.00      0.00      0.00      1490

    accuracy                           0.32      4755
   macro avg       0.11      0.33      0.16      4755
weighted avg       0.10      0.32      0.15      4755



***answer:***
- It seems that lemmatization has a small effect on model performance, but it is negligible.
- All POS have some influence on the sentiment, but it seems that combination of POS yields the only good results. POS on their own perform very badly.

## Part II: scikit-learn assignments
### [4 points] Question 5
Train the scikit-learn classifier (Naive Bayes) using the airline tweets.

+ Train the model on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=2)
+ Train with different settings:
    + with respect to vectorizing: TF-IDF ('airline_tfidf') vs. Bag of words representation ('airline_count') 
    + with respect to the frequency threshold (min_df). Carry out experiments with increasing values for document frequency (min_df = 2; min_df = 5; min_df =10) 
* [1 point] a. Generate a classification_report for all experiments
* [3 points] b. Look at the results of the experiments with the different settings and try to explain why they differ: 
    + which category performs best, is this the case for any setting?
    + does the frequency threshold affect the scores? Why or why not according to you?

In [97]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

def create_count_vec(tweets, min_df=2):
    vec = CountVectorizer(min_df=min_df,
                                 tokenizer=nltk.word_tokenize, 
                                 stop_words=stopwords.words('english'))
    count_vec = vec.fit_transform(tweets.data)
    return count_vec

def create_tfidf_vec(tweets, min_df=2):
    count_vec = create_count_vec(tweets, min_df)

    tfidf_transformer = TfidfTransformer()
    tfidf_vec = tfidf_transformer.fit_transform(count_vec)
    return tfidf_vec

#create vectors
vec_2 = create_count_vec(airline_tweets_train)
vec_5 = create_count_vec(airline_tweets_train, min_df=5)
vec_10 = create_count_vec(airline_tweets_train, min_df=10)

tfidf_vec_2 = create_tfidf_vec(airline_tweets_train)
tfidf_vec_5 = create_tfidf_vec(airline_tweets_train, min_df=5)
tfidf_vec_10 = create_tfidf_vec(airline_tweets_train, min_df=10)

def train_and_test_nb(vec, tweets):
    #split data
    docs_train, docs_test, y_train, y_test = train_test_split(
        vec, 
        tweets.target, 
        test_size = 0.2
        ) 
    #train
    clf = MultinomialNB().fit(docs_train, y_train)
    y_pred = clf.predict(docs_test)
    
    return classification_report(y_test, y_pred)

/Users/sebastiaanvergunst/anaconda3/envs/text_mining/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/sebastiaanvergunst/anaconda3/envs/text_mining/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/sebastiaanvergunst/anaconda3/envs/text_mining/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the st

In [99]:
print('Bag of Words, min_df=2:\n', train_and_test_nb(vec_2, airline_tweets_train), '\n')
print('Bag of Words, min_df=5:\n', train_and_test_nb(vec_5, airline_tweets_train), '\n')
print('Bag of Words, min_df=10:\n', train_and_test_nb(vec_10, airline_tweets_train), '\n')
print('---------------------------------------------------------------------\n')
print('TF-IDF, min_df=2:\n', train_and_test_nb(tfidf_vec_2, airline_tweets_train), '\n')
print('TF-IDF, min_df=5:\n', train_and_test_nb(tfidf_vec_5, airline_tweets_train), '\n')
print('TF-IDF, min_df=10:\n', train_and_test_nb(tfidf_vec_10, airline_tweets_train), '\n')

Bag of Words, min_df=2:
               precision    recall  f1-score   support

           0       0.85      0.94      0.89       358
           1       0.86      0.74      0.79       310
           2       0.84      0.86      0.85       283

    accuracy                           0.85       951
   macro avg       0.85      0.84      0.84       951
weighted avg       0.85      0.85      0.85       951
 

Bag of Words, min_df=5:
               precision    recall  f1-score   support

           0       0.82      0.90      0.86       329
           1       0.84      0.75      0.79       320
           2       0.83      0.84      0.83       302

    accuracy                           0.83       951
   macro avg       0.83      0.83      0.83       951
weighted avg       0.83      0.83      0.83       951
 

Bag of Words, min_df=10:
               precision    recall  f1-score   support

           0       0.83      0.91      0.87       334
           1       0.85      0.76      0.80      

***answer***:
- The Bag of Words, with min_df=2 performs best out of all, but the differences are not that big. Overall the min_df=2 setting seems to perform best and Bag of Words seems to outperform TF-IDF slightly.


### [4 points] Question 6: Inspecting the best scoring features 

+ Train the scikit-learn classifier (Naive Bayes) model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
* [1 point] a. Generate the list of best scoring features per class (see function **important_features_per_class** below) [1 point]
* [3 points] b. Look at the lists and consider the following issues: 
    + [1 point] Which features did you expect for each separate class and why?
    + [1 point] Which features did you not expect and why ? 
    + [1 point] The list contains all kinds of words such as names of airlines, punctuation, numbers and content words (e.g., 'delay' and 'bad'). Which words would you remove or keep when trying to improve the model and why? 

In [94]:
def important_features_per_class(vectorizer,classifier,n=80):
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:

#vec
airline_vec = CountVectorizer(min_df=2,
                             tokenizer=nltk.word_tokenize, 
                             stop_words=stopwords.words('english'))
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

#split data
docs_train, docs_test, y_train, y_test = train_test_split(
    airline_counts, 
    airline_tweets_train.target, 
    test_size = 0.2
    ) 

#train
clf = MultinomialNB().fit(docs_train, y_train)
important_features_per_class(airline_vec, clf)

/Users/sebastiaanvergunst/anaconda3/envs/text_mining/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Important words in negative documents
0 1512.0 @
0 1389.0 united
0 1253.0 .
0 416.0 ``
0 399.0 ?
0 395.0 flight
0 381.0 !
0 329.0 #
0 212.0 n't
0 159.0 ''
0 132.0 's
0 116.0 service
0 114.0 :
0 104.0 virginamerica
0 95.0 bag
0 93.0 get
0 91.0 delayed
0 88.0 cancelled
0 86.0 customer
0 84.0 ...
0 81.0 plane
0 77.0 time
0 77.0 ;
0 75.0 'm
0 74.0 -
0 72.0 hours
0 70.0 http
0 70.0 &
0 66.0 gate
0 64.0 airline
0 61.0 late
0 61.0 2
0 60.0 still
0 60.0 amp
0 57.0 hour
0 55.0 help
0 54.0 would
0 54.0 worst
0 52.0 one
0 51.0 flights
0 51.0 ca
0 50.0 waiting
0 50.0 like
0 47.0 never
0 45.0 flightled
0 44.0 (
0 43.0 us
0 43.0 fly
0 42.0 ever
0 42.0 )
0 41.0 've
0 40.0 due
0 40.0 back
0 38.0 wait
0 38.0 check
0 37.0 really
0 37.0 delay
0 37.0 $
0 36.0 seat
0 35.0 u
0 35.0 luggage
0 35.0 lost
0 35.0 day
0 35.0 3
0 34.0 trying
0 34.0 last
0 34.0 bags
0 33.0 seats
0 33.0 got
0 33.0 going
0 33.0 crew
0 32.0 thanks
0 32.0 people
0 32.0 guys
0 32.0 baggage
0 31.0 airport
0 30.0 ticket
0 30.0 terrible
0 

***answer:***
- would expect the '@', since this is included because it's concerning a tweet. For the purpose of finding the most important features this can be omitted, since it doesn't add any value (TF_IDF takes care of this).
- 'delayed' and 'cancelled' are in the neutral category, which I would expect in the negative category, since a flight being cancelled or delayed is not a nice experience.
- Would remove the airline names, you don't want to train the model on the airline names, since this makes the model biased. Would also remove some of the punctuation, but not all. Seems like the exclamation mark is more used in positive reviews and could be useful. 

### [Optional! (will not  be graded)] Question 7
Train the model on airline tweets and test it on your own set of tweets
+ Train the model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
+ Apply the model on your own set of tweets and generate the classification report
* [1 point] a. Carry out a quantitative analysis.
* [1 point] b. Carry out an error analysis on 10 correctly and 10 incorrectly classified tweets and discuss them
* [2 points] c. Compare the results (cf. classification report) with the results obtained by VADER on the same tweets and discuss the differences.

### [Optional! (will not be graded)] Question 8: trying to improve the model
* [2 points] a. Think of some ways to improve the scikit-learn Naive Bayes model by playing with the settings or applying linguistic preprocessing (e.g., by filtering on part-of-speech, or removing punctuation). Do not change the classifier but continue using the Naive Bayes classifier. Explain what the effects might be of these other settings 
+ [1 point] b. Apply the model with at least one new setting (train on the airline tweets using 80% training, 20% test) and generate the scores
* [1 point] c. Discuss whether the model achieved what you expected.

## End of this notebook